<a href="https://colab.research.google.com/github/MRK4863/healthcare_chat_asst/blob/main/google_colab/graphRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neo4j -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 7.9 MB/s eta 0:00:00


In [9]:
import pandas as pd
from neo4j import GraphDatabase
from itertools import combinations

In [10]:
# Read from environment variables or .env file
uri = "neo4j+s://fecbe539.databases.neo4j.io"
username = "neo4j"
password = "yaxMZJyX9a6Ph4SNdRncWJWbSWbmiNGS9Flah_UV100"

In [11]:
# Connect to Neo4j Aura
driver = GraphDatabase.driver(
    uri,
    auth=("neo4j", password)
)

In [12]:
df_drug_db = pd.read_csv("/content/drugbank_clean.csv")

filtered_df = df_drug_db.dropna(subset=['name', 'drugbank-id'])

name_to_drugbank_dict = dict(zip(filtered_df['name'], filtered_df['drugbank-id']))

<ipython-input-12-3289901911>:1: DtypeWarning: Columns (29,30,31,32,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_drug_db = pd.read_csv("/content/drugbank_clean.csv")


In [13]:
def get_drug_info(tx, drug_names):
    query = """
    MATCH (d:Drug)
    WHERE d.name IN $drug_names
    RETURN d.drugbank_id AS id, d.name AS name, d.description AS description,
           d.indication AS indication, d.mechanism_of_action AS mechanism, d.toxicity AS toxicity, d.food_interactions as food_interactions
    """
    return list(tx.run(query, drug_names=drug_names))

def get_interactions(tx, pairs):
    results = []
    for drug1, drug2 in pairs:
        # drug_1_id, drug_2_id = name_to_drugbank_dict[drug1], name_to_drugbank_dict[drug2]
        query = """
        MATCH (a:Drug {name: $drug1})-[r:INTERACTS_WITH]-(b:Drug {name: $drug2})
        RETURN a.name AS drug1, b.name AS drug2, r.description AS description
        """
        rec = tx.run(query, drug1=drug1, drug2=drug2).single()
        if rec:
            results.append(dict(rec))
    return results

drug_list = ["Titanium dioxide", "Verteporfin", "Trioxsalen", "Cetirizine", "Acetaminophen"]
pairs = list(combinations(drug_list, 2))

with driver.session() as session:
    drug_infos = session.read_transaction(get_drug_info, drug_list)
    interaction_infos = session.read_transaction(get_interactions, pairs)

<ipython-input-13-4113845751>:27: DeprecationWarning: read_transaction has been renamed to execute_read
  drug_infos = session.read_transaction(get_drug_info, drug_list)
<ipython-input-13-4113845751>:28: DeprecationWarning: read_transaction has been renamed to execute_read
  interaction_infos = session.read_transaction(get_interactions, pairs)


In [14]:
def build_prompt(drug_infos, interaction_infos):
    """
    Build a well-organized, markdown-formatted prompt for drug information analysis.

    Args:
        drug_infos (list): List of dictionaries containing drug information
        interaction_infos (list): List of dictionaries containing drug interaction information

    Returns:
        str: Formatted prompt string
    """

    prompt = """# Clinical Drug Analysis Assistant

## Instructions
You are a clinical assistant tasked with analyzing drug information. Please follow these steps:

1. **Summarize each drug** - Provide a clear, concise summary of each medication
2. **Analyze combination risks** - List and explain the risks of combining any of these drugs
3. **Base responses on provided data only** - Do not generate information that cannot be verified from the data below

---

## Drug Information Database

"""

    # Add individual drug information
    for i, drug in enumerate(drug_infos, 1):
        prompt += f"""### {i}. {drug['name']}

**Description:** {drug['description']}

**Clinical Details:**
- **Indication:** {drug['indication']}
- **Mechanism of Action:** {drug['mechanism']}
- **Toxicity Profile:** {drug['toxicity']}
- **Food Interactions:** {drug['food_interactions']}

---

"""

    # Add drug interaction information
    prompt += """## Drug-Drug Interaction Analysis

"""

    if interaction_infos and len(interaction_infos) > 0:
        prompt += """### Known Interactions

"""
        for i, interaction in enumerate(interaction_infos, 1):
            prompt += f"""**{i}. {interaction['drug1']} + {interaction['drug2']}**
- Risk: {interaction['description']}

"""
    else:
        prompt += """### No Known Interactions
No documented drug-drug interactions were found among the provided medications based on the available data.

"""

    prompt += """---

## Output Requirements

Please structure your response as follows:

### Drug Summaries
Provide a concise summary for each medication listed above.

### Combination Risk Assessment
Analyze and explain any risks associated with combining these medications, based solely on the interaction data provided.

### Clinical Recommendations
Offer appropriate guidance for patients or clinicians based on the available information. Include food-interaction recommendations as well if appropriate

**Note:** All information should be clearly attributed to the provided data sources. Do not include speculative or unverified information."""

    return prompt

prompt = build_prompt(drug_infos, interaction_infos)


In [15]:
print(prompt)

# Clinical Drug Analysis Assistant

## Instructions
You are a clinical assistant tasked with analyzing drug information. Please follow these steps:

1. **Summarize each drug** - Provide a clear, concise summary of each medication
2. **Analyze combination risks** - List and explain the risks of combining any of these drugs
3. **Base responses on provided data only** - Do not generate information that cannot be verified from the data below

---

## Drug Information Database

### 1. Acetaminophen

**Description:** Acetaminophen (paracetamol), also commonly known as _Tylenol_, is the most commonly taken analgesic worldwide and is recommended as first-line therapy in pain conditions by the World Health Organization (WHO).[A176318] It is also used for its antipyretic effects, helping to reduce fever.[F4124] This drug was initially approved by the U.S. FDA in 1951 and is available in a variety of forms including syrup form, regular tablets, effervescent tablets, injection, suppository, and ot

In [16]:
from openai import OpenAI

client = OpenAI(
    api_key="f9c890ca-64fa-4e37-ab62-fd9a1e6c4de6",
    base_url="https://api.sambanova.ai/v1",
)

response = client.chat.completions.create(
    model="Meta-Llama-3.3-70B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful clinical assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.1,
    top_p=0.1
)

print(response.choices[0].message.content)


### Drug Summaries

1. **Acetaminophen**: A widely used analgesic and antipyretic, available over-the-counter, used for mild to moderate pain and fever reduction. It has a low risk of allergic reactions but can cause liver toxicity in overdose.

2. **Cetirizine**: An orally active second-generation histamine H1 antagonist, effective in treating allergic symptoms such as sneezing, coughing, and nasal congestion, with minimal sedating effects.

3. **Verteporfin**: A benzoporphyrin derivative used in photodynamic therapy to treat abnormal blood vessels in conditions like macular degeneration. It accumulates in these vessels and, when activated by light, produces reactive oxygen species that damage the endothelium.

4. **Trioxsalen**: A furanocoumarin derivative used in conjunction with UV-A light for the treatment of vitiligo and hand eczema. It causes photosensitization of the skin and, upon activation, creates interstrand cross-links in DNA, leading to cell apoptosis.

5. **Titanium dio